In [ ]:
# 🔧 Install required packages
!pip install paddleocr
!pip install paddlepaddle

# 📦 Import Libraries
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
import cv2
import numpy as np
import os
import base64
import shutil
from paddleocr import PaddleOCR, draw_ocr
import matplotlib.pyplot as plt

# 📸 Webcam capture function
def capture_image():
    js = Javascript('''
        async function takePhoto() {
            const div = document.createElement('div');
            const capture = document.createElement('button');
            capture.textContent = '📸 Capture';
            div.appendChild(capture);

            const video = document.createElement('video');
            video.style.display = 'block';
            const stream = await navigator.mediaDevices.getUserMedia({video: true});
            document.body.appendChild(div);
            div.appendChild(video);
            video.srcObject = stream;
            await video.play();

            google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);
            await new Promise((resolve) => capture.onclick = resolve);

            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            stream.getTracks().forEach(t => t.stop());
            div.remove();
            return canvas.toDataURL('image/jpeg');
        }
        takePhoto();
    ''')

    display(js)
    data = eval_js('takePhoto()')
    data = data.split(',')[1]
    img_bytes = base64.b64decode(data)
    np_arr = np.frombuffer(img_bytes, dtype=np.uint8)
    img = cv2.imdecode(np_arr, cv2.IMREAD_COLOR)
    return img

# 🧠 Load Haar Cascade for plate detection
cascade_path = '/content/model/haarcascade_russian_plate_number.xml'  # Adjust path if needed
plate_cascade = cv2.CascadeClassifier(cascade_path)

if plate_cascade.empty():
    print("❌ Cascade file failed to load. Check the filename or re-upload.")
else:
    print("✅ Cascade loaded successfully.")

# 🔍 Plate Detection and Saving
def detect_and_save_plate(img, cascade_path="model/haarcascade_russian_plate_number.xml"):
    os.makedirs("plates", exist_ok=True)
    plate_cascade = cv2.CascadeClassifier(cascade_path)

    # Shift existing files up
    existing_files = sorted(
        [f for f in os.listdir("plates") if f.startswith("scaned_img_") and f.endswith(".jpg")],
        key=lambda x: int(x.split('_')[-1].split('.')[0]),
        reverse=True
    )

    for f in existing_files:
        current_index = int(f.split('_')[-1].split('.')[0])
        new_name = f"scaned_img_{current_index + 1}.jpg"
        shutil.move(os.path.join("plates", f), os.path.join("plates", new_name))

    # Detect plates
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    plates = plate_cascade.detectMultiScale(img_gray, 1.1, 4)
    min_area = 500
    saved = False

    for (x, y, w, h) in plates:
        area = w * h
        if area > min_area:
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(img, "Number Plate", (x, y - 5), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (255, 0, 255), 2)

            img_roi = img[y:y + h, x:x + w]
            save_path = "plates/scaned_img_0.jpg"
            cv2.imwrite(save_path, img_roi)
            print(f"✅ Latest plate saved to: {save_path}")
            cv2_imshow(img_roi)
            saved = True
            break

    if not saved:
        print("❌ No plate detected.")
    cv2_imshow(img)

# 🚀 Run Full Pipeline
print("📸 Capturing image from webcam...")
img = capture_image()

print("🔍 Detecting license plate...")
detect_and_save_plate(img)

# 🧠 OCR with PaddleOCR
ocr = PaddleOCR(use_angle_cls=True, lang='en')  # English OCR
result = ocr.ocr('/content/plates/scaned_img_0.jpg', cls=True)

# 📤 Extract and Print Plate Text
plate_text = ''
if result and result[0]:
    plate_text = ''.join([line[1][0] for line in result[0]]).replace(" ", "").upper()
    print("🚗 The car number plate is:", plate_text)
else:
    print("❌ No text detected on plate.")

# 🕵️‍♂️ Mock stolen plate database check
stolen_plates_db = ["21BH2345AA", "XY987ZT", "MH12DE1433"]

if plate_text in stolen_plates_db:
    print(f"🚨 ALERT: Vehicle with plate '{plate_text}' is reported STOLEN!")
else:
    print(f"✅ Vehicle with plate '{plate_text}' is clear.")
